In [ ]:
import xgboost as xgb
def get_xgb_model(n_tree, n_jobs=20):
    model = xgb.XGBClassifier(
        n_estimators=n_tree,
        max_depth=3,
        min_child_weight=300,
        learning_rate=0.1,
        n_jobs=n_jobs,
        tree_method='hist',
    )
    return model

In [ ]:
xgb_model_A = get_xgb_model(75)

In [ ]:
def run_model(df_train,df_val,df_test,model_obj,model_feat_colms):
    
    # make vectors
    y_val = df_val.label.to_numpy()
    X_val = df_val[model_feat_colms].to_numpy()

    # Make Train Vectors
    y_train = df_train.label.to_numpy()
    X_train = df_train[model_feat_colms].to_numpy()
    
    # Make Test Vectors
    y_test = df_test.label.to_numpy()
    X_test = df_test[model_feat_colms].to_numpy()

     # train the model
    model_obj.fit(X = X_train, y=y_train)
    
    # Predict
    y_pred_val = model_obj.predict_proba(X_val)
    y_pred_train = model_obj.predict_proba(X_train)
    y_pred_test= model_obj.predict_proba(X_test)

    # performance
    auc_train = roc_auc_score(y_train,y_pred_train[:,1])
    auc_val = roc_auc_score(y_val,y_pred_val[:,1])
    auc_test = roc_auc_score(y_test,y_pred_test[:,1])

    df_val["score"] =y_pred_val[:,1]
    df_train["score"]=y_pred_train[:,1]
    df_test["score"]=y_pred_test[:,1]
    # Feature Importances
    feat_imp = list(zip(model_feat_colms, model_obj.feature_importances_))
    return (auc_train, auc_test, auc_val), feat_imp

In [ ]:
auc_scores_A,feat_imp = run_model(model_obj=xgb_model_A,
                                model_feat_colms=filtered_features,
                                df_train=df_train,
                                df_val=df_val,
                                 df_test = df_test)
auc_scores_A

In [ ]:
def print_metrics(true_vec, pred_vec, this_deciles, cut_off=0.5):
    """
    """
    n_deciles = len(this_deciles) - 1
    pred_deciles = pd.cut(
        np.squeeze(pred_vec), 
        this_deciles, 
        labels = range(n_deciles)
    )
    
    pred_vec_cls = (pred_vec>cut_off).astype('int')
    precision = precision_score(true_vec, pred_vec_cls)
    recall = recall_score(true_vec, pred_vec_cls)
    auc = roc_auc_score(true_vec, pred_vec)
    accuracy = accuracy_score(true_vec, pred_vec_cls)
    decile_matrix = pd.crosstab(true_vec, pred_deciles)
    conf_matrix = pd.crosstab(true_vec, pred_vec_cls)
    bad_rate = pd.crosstab(true_vec, pred_deciles, normalize='columns').to_numpy()[1:,][0]
    
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1: {2*recall*precision/(recall+precision)}')
    print(f'Accuracy: {accuracy}')
    print(f'AUC: {auc}')
    print(f'Confusion_matrix:\n{conf_matrix}')
    print(f'Decile_matrix:\n{decile_matrix}')
    print(f'Bad Rate:\n{bad_rate}')

In [ ]:
decile_cuts_all_apr = [
    0.0,
    0.534209132194519,
    0.569546103477478,
    0.5933143496513367,
    0.6123031973838806,
    0.6289125084877014,
    0.6502689123153687,
    0.6742594540119169,
    0.6988576054573059,
    0.7518265485763549,
    1.0
]
print_metrics(y_train, y_pred_tr, decile_cuts_all)